In [ ]:
import numpy as np
import os
import cv2
from imutils import paths
import shutil

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.cm import coolwarm

from keras.preprocessing import image
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

import itertools

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# function for feature extraction using VGG16 CNN
def extract_features_vgg16(img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    out = model.predict(x)
    return out

# Initialize keras model for VGG16 architecture
model = vgg16.VGG16(include_top=False,weights='imagenet',pooling = 'avg')

OUTPUT_DATASET =  "Cropped_Images_SEN"
FOL = "SEN"
RESULTS = "Results"
NImages = ["NX_NY_Images_1", "length_125","length_100","NX_NY_Images_2","NX_NY_Images_3","NX_NY_Images_4",
          "NX_NY_Images_5","NX_NY_Images_6","NX_NY_Images_7",
           "NX_NY_Images_8", "NX_NY_Images_9","NX_NY_Images_10"]

L = 0
for split in NImages:
    print("[INFO] processing '{} folder'...".format(split))
    p= os.path.sep.join([OUTPUT_DATASET, split])
    results_DIR = os.path.sep.join([FOL, RESULTS, split])
    if not os.path.exists(results_DIR):
        os.makedirs(results_DIR)
    imagePaths = list(paths.list_images(p))
    totalImages = len(list(paths.list_images(p)))
    if totalImages == 0:
        pass
    else:
        features = np.empty([totalImages,512])    
        labels_num = []
        i=0
        for file in imagePaths:
            filename = file.split(os.path.sep)[-1]
            curr_label1 = filename.split('_')[0]
            curr_label2 = filename.split('_')[1]
            if curr_label1 == "DF140T":
                labels_num.append(0)
            elif curr_label1 == "DP980":
                labels_num.append(1)
                
            img = image.load_img(file, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features[i] = model.predict(x)
            i+=1    

        totalImages = len(features)
        if totalImages < 50:
            n_comp = totalImages
        else:
            n_comp = 50
        # PCA dimensionality reduction to 50 components
        pca_model = PCA(n_components=n_comp)
        x_pca = pca_model.fit_transform(features) 
        
        
        colors = [0, 1]
        labels_name = ['DF140TSEN', 'DP980SEN']
        for p in range(5, 60, 5):

            print('\n')
            print('Perplexity = {}'.format(p))
            print('\n')
            
            # Take the reduced features by PCA and insert them into t-sne
            x_tsne = TSNE(n_components=2, perplexity=p, learning_rate=200.0, n_iter=3000, verbose=1).fit_transform(x_pca)

            plt.rcParams['font.size'] = 14
            fig1 = plt.figure(figsize=(6, 4))
            ax1 = plt.axes(frameon=True)
            plt.setp(ax1, xticks=(), yticks=())
            plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                                wspace=0.0, hspace=0.0)
            scatter1 = plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=labels_num, cmap="brg")
            handles1 = [plt.plot([],color=scatter1.get_cmap()(scatter1.norm(c)),ls="", marker="o")[0] for c in colors ] 
            plt.tight_layout()
            scatter1 .set_clip_on(False) 
            legend1 = ax1.legend(handles1, labels_name, loc='best')
            ax1.add_artist(legend1)
            out_name = 'TSNE_perplexity_' + str(p) + '_.png'
            plt.savefig(os.path.sep.join([results_DIR, out_name]))
            plt.close()                            
            
            # -------------------------------------------------------------------------------------------------------------------------
            #   Kmeans labeling
            #--------------------------------------------------------------------------------------------------------------------------
            
            kmeans = KMeans(n_clusters=2, init='k-means++', n_init=30, max_iter=1000, 
                            tol=0.0001, precompute_distances='deprecated', verbose=0, random_state=None, 
                            copy_x=True, n_jobs='deprecated', algorithm='auto')
            kmeans.fit(x_tsne)
            labs = kmeans.labels_
            target_array = np.asarray(labels_num,dtype=np.int8)
            correspond_labels = np.zeros(labs.shape)
            target = np.array(labels_num)
            
            # Convert the k-Means labeling into the same colormap labeling as the ground truth labeling
            correspond_labels[labs==0] = np.argmax(np.bincount(target_array[labs==0]))    
            correspond_labels[labs==1] = np.argmax(np.bincount(target_array[labs==1]))
                                        
            # find the errors
            l = [k if t==k else max(target)+1 for t,k in zip(target, correspond_labels)]
            l = np.asarray(l,dtype=np.int8)
        
            cor_idx = l<=max(labels_num)
            error_idx = l==max(labels_num)+1
        
            list_labels = l[cor_idx]
            error_labels = l[error_idx]
            
            # -------------------------------------------------------------------------------------------------
            #  Plot kmeans labeling
            # -------------------------------------------------------------------------------------------------
                            
            # Plot the result with scatter
            figx = plt.figure(figsize=(6, 4))
            axx = plt.axes(frameon=True)
            plt.setp(axx, xticks=(), yticks=())
            plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9, wspace=0.0, hspace=0.0)
            scatterx = plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=correspond_labels, cmap="brg")
            handlesx = [plt.plot([],color=scatterx.get_cmap()(scatterx.norm(c)),ls="", marker="o")[0] for c in colors ] 
            plt.tight_layout()
            scatterx .set_clip_on(False) 
            legendx = axx.legend(handlesx, labels_name,  loc='best')
            axx.add_artist(legendx)
            
            out_name = 'Kmeans_labeled_perplexity_' + str(p) + '_.png'
            plt.savefig(os.path.sep.join([results_DIR, out_name]))
            plt.close()  

            # -------------------------------------------------------------------------------------------------
            #  Plot kmeans labeling with erros
            # -------------------------------------------------------------------------------------------------
                
            # Plot the result with scatter
            fig3 = plt.figure(figsize=(6, 4))
            ax3 = plt.axes(frameon=True)
            plt.setp(ax3, xticks=(), yticks=())
            plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9, wspace=0.0, hspace=0.0)
            sc1 = plt.scatter(x_tsne[cor_idx, 0], x_tsne[cor_idx, 1], c=list_labels, cmap="brg")
            sc2 = plt.scatter(x_tsne[error_idx, 0], x_tsne[error_idx, 1], c="k", marker = "x" )
            handles3 = [plt.plot([],color=sc1.get_cmap()(sc1.norm(c)),ls="", marker="o")[0] for c in colors ]   
            plt.tight_layout()
            sc1 .set_clip_on(False) 
            sc2 .set_clip_on(False) 
            legend3 = ax3.legend(handles3, labels_name,  loc='best')
            ax3.add_artist(legend3)

            out_name = 'Kmeans_errors_' + str(p) + '_.png'
            plt.savefig(os.path.sep.join([results_DIR, out_name]))
            plt.close()

            # -------------------------------------------------------------------------------------------------
            #  Confusion Matrix
            # -------------------------------------------------------------------------------------------------

            # Compute confusion matrix
            cm = confusion_matrix(target_array, correspond_labels)
            np.set_printoptions(precision=2)
                
            # Plot normalized confusion matrix
            plt.figure(figsize=(6,6)) 
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
            tick_marks = np.arange(len(labels_name))
            plt.xticks(tick_marks, labels_name, rotation=45)
            plt.yticks(tick_marks, labels_name)
        
            fmt = '.2f'
            thresh = cm.max() / 2.
            for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
                plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
        
            plt.tight_layout()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            
            out_name = 'Confusion_Mat_perplexity_' + str(p) + '_.png'
            plt.savefig(os.path.sep.join([results_DIR, out_name]))
            plt.close()
            
            if L==0:
              output_txt =  os.path.sep.join([FOL, RESULTS, "Accuracy.txt"])
              text_file = open(output_txt, "a")
              text_file.write('{:s}, {:s} , {:s} \n'.format('Length_Scale', 'Perplexity', 'Clustering Accuracy'))

            text_file = open(output_txt, "a")
            text_file.write('{:s}, {:d} ,{:.3f} \n'.format(Length_Scale[L], p, accuracy_score(target_array, correspond_labels)))
            text_file.close()
        L+=1